# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [209]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [210]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [211]:
wine_df.shape[0]



178

_(ii)_ How many variables (columns) does the dataset contain?

In [213]:
wine_df.shape[1]


14

_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [215]:

wine_df['class'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 178 entries, 0 to 177
Series name: class
Non-Null Count  Dtype
--------------  -----
178 non-null    int32
dtypes: int32(1)
memory usage: 844.0 bytes


In [216]:
wine_df['class'].unique()

array([0, 1, 2])


_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [217]:

wine_df.describe()


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [219]:

predictors_wine_df = wine_df.drop(columns=['class'])


In [220]:
predictors_wine_df.shape[1]

13

You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [221]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

> YoIn k-nearest neighbors (KNN), the scale of the features matters significantly because KNN relies on distance calculations to classify data points. Features with larger ranges can dominate the distance calculations, leading to biased results. To address this issue, it is important to standardize the data.
By standardizing the features to have a mean of zero and a standard deviation of one (using tools like StandardScaler()), you put all variables on the same scale. This ensures that no feature disproportionately influences the KNN classification results, which leads to more balanced and meaningful comparisons across features.


(ii) Why did we elect not to standard our response variable `Class`?


Because it is a categorical variable representing different categories or labels (e.g., "yes" or "no", "A" or "B") even though it is represented by numerical values like 0, 1, 2. Standardization is not necessary for such categorical values because they are used as labels for classification, not for measuring distances or other mathematical operations.


(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

In [222]:
# Do not touch
random.seed(123)

Setting a random seed is a way to ensure that operations involving randomness produce the same results each time they are run. This is important for reproducibility, which allows us to verify and validate the results at a later time.

The specific value of a random seed itself is not inherently important. Any value can be used, and it will ensure reproducibility of your results. The key aspect is consistency—using the same random seed will always give you the same result, while using different seeds will lead to different outputs.

However, there are a few considerations:

Reproducibility: It's better to choose any fixed seed to ensure reproducibility. It doesn't matter if we use 0, 42, or 1234; as long as we use the same value every time, our results will be reproducible.

Exploration: If you want to test the robustness of our model or approach, we can try using different seeds to see how sensitive the results are to randomness. This can help identify whether ourr model performance depends heavily on how data was split or initialized.

Convention: Some values, like 42, are popular because of cultural references, but they do not have any technical advantage over other numbers.

I was at the Seminar of Dr. Jafari Jozani from University of Manitoba and he suggested that in the case of KNN or in Cross Validation it is important that check the different seeds to examine that we get the same result, it means the model is good in any random set or not.


(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [224]:
# Create a random vector of True and False values to split the data
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

split

# Create training and test sets for predictors 
predictors_train = predictors_standardized[split]  # Use true of 'split' to get train data

predictors_test = predictors_standardized[~split]  # Use the opposite of 'split' to get test data

# Create traing and test sets for response variables 
#response_test = response_variable[~split]

response_variable_train = wine_df.iloc[split, -1] # Select 'class' , last column , for response variable
response_variable_test = wine_df.iloc[~split, -1]

print(response_variable_train.shape[0])
print(response_variable_test.shape[0])
print(predictors_test)
print(response_variable_test)   

142
36
      alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
1    0.246290   -0.499413 -0.827996          -2.490847   0.018145   
3    1.691550   -0.346811  0.487926          -0.809251   0.930918   
16   1.605081   -0.373741  1.292101           0.151661   1.422412   
20   1.308617   -0.634063 -0.316249          -1.049479   1.843692   
23  -0.186053   -0.660993  0.561033          -0.508966  -0.332922   
29   1.259207   -0.589180 -0.572122          -1.049479  -0.262708   
33   0.938038   -0.723829  1.218995           0.001518   2.264972   
43   0.295700    1.475446 -0.279695          -0.599052   0.228785   
56   1.506260   -0.571226 -0.243142          -0.959394   1.281985   
61  -0.445459   -0.876432 -1.266637          -0.809251   0.018145   
66   0.135116   -1.190614 -2.436346          -1.349764  -1.526548   
72   0.604516   -0.607133 -0.462462           1.352801  -0.894628   
74  -1.285439   -1.118801 -0.243142           0.451946   0.088358   
79  -0.371343    1.376703  

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [226]:
knn = KNeighborsClassifier()
knn

KNeighborsClassifier()

In [244]:
parameter_grid = {
    "n_neighbors": range(1, 51, 5),
}
wine_tune_grid = GridSearchCV(
    estimator=knn,
    param_grid=parameter_grid,
    cv=10
)
wine_tune_grid.fit( predictors_train, response_variable_train)

accuracies_grid = pd.DataFrame(wine_tune_grid.cv_results_)
accuracies_grid

wine_tune_grid.best_params_


{'n_neighbors': 31}

#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [245]:
knn = KNeighborsClassifier(n_neighbors=31)
knn

KNeighborsClassifier(n_neighbors=31)

In [246]:
knn.fit(predictors_train, response_variable_train)
predictions_wine = knn.predict(predictors_test)
predictions_wine

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [247]:
knn.score(predictors_test , response_variable_test)

0.9722222222222222

In [248]:
pd.crosstab(response_variable_test,predictions_wine)

precision_score(
    y_true=response_variable_test,
    y_pred=predictions_wine,
    pos_label="1" ## Calculate precision  for class 1 (considering class 1 as the positive class) or we can change for class 2
    ,average='weighted'
)

c:\Users\user\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1583: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


0.9750000000000001

In [243]:

recall_score(
    y_true=response_variable_test,
    y_pred=predictions_wine,
    pos_label="1" ## Calculate recall for class 1 (considering class 1 as the positive class)or we can change for class 2
    ,average='weighted'
)

c:\Users\user\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1583: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


0.9722222222222222

# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
